In [1]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 5)

#from matplotlib import pyplot as plt
from util import util
import igraph as ig
import numpy as np

# ============================================================
# Build a few reference graphs
# ============================================================

# The small graph used to explain concepts
eoh = 4
g = util.build_website_graph(nnodes=4, rate=3, extra_arc_fraction=0.25, seed=42)
flows, paths = util.build_random_paths(g, min_paths=3, max_paths=5,
                                          min_units=1, max_units=10, eoh=eoh, seed=42)
tug = util.build_time_unfolded_graph(g, eoh=eoh)
node_counts, arc_counts = util.get_counts(tug, flows, paths)

# From Pricing...

Because there's an elephant in the room, and it's HUGE

## Scalability, or Lack Thereof

**Our current approach as one, massive, limitations**

The number paths in graph scales _exponentially_ on its size

* Meaning that path enumeration becomes quickly very expensive
* ...And the path formulation size grows at the same rate

**Let's check the solution time for our small example graph:**

In [3]:
%time rflows, rpaths = util.solve_path_selection_full(tug, node_counts, arc_counts, verbose=0)

CPU times: user 144 ms, sys: 1.98 ms, total: 146 ms
Wall time: 145 ms


...And the for a slightly larger graph (8 nodes, 5 time steps):

In [3]:
g8_5, t8_5, f8_5, p8_5, nc8_5, ac8_5 = util.get_default_benchmark_graph(nnodes=8, eoh=5, seed=42)
%time f8_5, p8_5 = util.solve_path_selection_full(t8_5, nc8_5, ac8_5, verbose=0)

CPU times: user 10.9 s, sys: 162 ms, total: 11 s
Wall time: 11 s


 ## Adding Variables on Demand
 
**What if we had a way to _add variables on demand_?**
 
Then could think of:

<center><img src="assets/cg_scheme.png" width=700px/></center>
 
* Solving the Path Formulation with a _subset of paths_
* ...Then searching for new paths to be added
  - If we find some, we add them to the pool and we repeat
  - If we find none, we are done

**An approach such as this may strongly _mitigate our scalability issues_**

## Adding Variables on Demand

**What do we need to pull this off?**

<center><img src="assets/cg_scheme.png" width=550px/></center>

1. The ability to use a limited pool in the path formulation
2. A way to identify new paths to be added

Point 1 is trivial, but what about point 2?

**We could split the enumeration in multiple "chunks"**

* That would allow to obtain the first solutions more quickly
* But would still need to complete the enumeration to prove optimality

**What we need is a way to identify _useful_ paths that are not yet in the pool**

## Identifying Useful Variables

**Let's recall the structure of the Path Formulation**

$$
\arg \min_{x} \left\{f(x) \mid x \geq 0 \right\}  \quad \text{with: } f(x) = \frac{1}{2} x^T P x + q^T x
$$

* We can view missing variables are having _value 0_ in the current solution
* So, we are looking for variables that can be _raised to reduce error_
* Since the problem is convex, we could start by _looking at the gradient_

**Hence, we could search for variables with a _negative gradient term_**

$$
\frac{\partial}{\partial x_j} f(x) < 0
$$

* This a _necessary_ condition in general
* Later in the course we will found out why

## Pricing Problem

**Let's revisit and generalize our schema**

<center><img src="assets/cg_scheme2.png" width=700px/></center>

* The "main" problem may not involve paths
  - ...So we will call it just a _master problem_
* We look for additional variables such that $\frac{\partial}{\partial x_j} f(x) < 0$
  - It's a bit like we are assigning a "price tag" to them
  - If the price is positive, we skip the variable (we know it's useless now)
  - If the price is negative, the variable _may_ be useful
  - For this reasons, we call the second component _pricing problem_

## The Path Selection Gradient

**We need to compute gradient terms for variables _not yet in the pool_**

This is easiest if we differentiate our original (equivalent) objective:
$$
f(x) = \frac{1}{2} \|Vx - \hat{v}\|_2^2 + \frac{1}{2} \|Ex - \hat{e}\|_2^2
$$

* ...Since node contribution mix-up in the $P$ matrix and $q$ vectors

**The least square objective can be rewritten as:**
$$
f(x) =  \frac{1}{2} \sum_{i = 1}^{n_v} \left(\sum_{j=1}^n V_{ij} x_{j} - \hat{v}_i\right)^2 +
\frac{1}{2} \sum_{k = 1}^{n_e} \left(\sum_{j=1}^n E_{kj} x_{j} - \hat{e}_k\right)^2
$$

* Where $n_v$ is the number of nodes and $n_e$ the number of arcs

## The Path Selection Gradient

**We can differentiate the expression to obtain**
$$
\frac{\partial}{\partial x_j} f(x) = \sum_{i=1}^{n_v} \left(\sum_{j=1}^n V_{ij} x_{j} - \hat{v}_i\right) V_{ij} +
\sum_{k=1}^{n_e} \left(\sum_{j=1}^n E_{kj} x_{j} - \hat{e}_k\right) E_{kj}
$$

Some expressions in the formula are simply the node/edge _residuals_:
$$
r^v_i = \sum_{j=1}^n V_{ij} x_{j} - \hat{v}_i \quad \text{and} \quad r^e_k = \sum_{j=1}^n E_{kj} x_{j} - \hat{e}_k
$$

Hence we can rewrite the gradient terms as:
$$
\frac{\partial}{\partial x_j} f(x) = \sum_{i=1}^{n_v} r^v_{i} V_{ij} +
\sum_{k=1}^{n_e} r^e_{k} E_{kj}
$$


## The Path Selection Gradient

**Now, let's parse the meaning of our gradient term:**

$$
\frac{\partial}{\partial x_j} f(x) = \sum_{i=1}^{n_v} r^v_{i} V_{ij} +
\sum_{k=1}^{n_e} r^e_{k} E_{kj}
$$

* For every (TUG) node $i$ included in the path, we add $r^v_i$
* For every (TUG) arc $k$ included in the path, we add $r^e_k$

**This is a simple computation that we can perform _on any path_**

...Including those that are not yet in the path formulation pool

* Just don't forget that this condition identifies (potentially) useful paths
* ...But only w.r.t. the current path formulation solution!

## A Look at the Residuals

**Let's try it out**

First, we enumerate all TUG paths

In [4]:
tugs, tugs_source = util._add_source_to_tug(tug)
tug_paths = util.enumerate_paths(tugs, tugs_source, exclude_source=True)

Then, we run the path formulation with a limited pool of paths

In [5]:
path_pool = tug_paths[:10]
rflows0, rpaths0 = util.solve_path_selection_full(tug, node_counts, arc_counts,
                                                  initial_paths=path_pool, verbose=0)
sse = util.get_reconstruction_error(tug, rflows0, rpaths0, node_counts, arc_counts)
util.print_solution(tug, rflows0, rpaths0, sort='descending')
print(f'RSSE: {np.sqrt(sse):.2f}')

1.96: 0,0 > 1,0 > 2,0 > 3,2
1.86: 0,0 > 1,0 > 2,0 > 3,3
0.79: 0,0 > 1,0 > 2,0 > 3,0
RSSE: 25.58


* Since we are restricted to a subset of paths, the RSSE is no longer 0

## A Look at the Residuals

**Then we can extract the residuals, i.e. $V x - \hat{v}$ and $E x - \hat{e}$**

In [6]:
nres0, ares0 = util._get_residuals(tug, rflows0, rpaths0, node_counts, arc_counts)
print('NODE RESIDUALS')
print('\t'.join(f'{k}:{v:.2f}' for k, v in nres0.items()))
print('ARC RESIDUALS')
print('\t'.join(f'{k}:{v:.2f}' for k, v in ares0.items()))

NODE RESIDUALS
(0, 0):4.61	(0, 1):-4.89	(0, 2):-5.47	(0, 3):0.00	(1, 0):1.29	(1, 1):-4.89	(1, 2):-5.47	(1, 3):0.00	(2, 0):-3.60	(2, 1):0.00	(2, 2):-5.47	(2, 3):-8.17	(3, 0):-4.10	(3, 1):0.00	(3, 2):-6.83	(3, 3):-10.05
ARC RESIDUALS
(1, 0, 0):4.61	(1, 0, 1):0.00	(1, 1, 1):-4.89	(1, 0, 2):0.00	(1, 2, 2):-5.47	(1, 0, 3):0.00	(1, 3, 3):0.00	(1, 1, 0):0.00	(1, 1, 2):0.00	(2, 0, 0):1.29	(2, 0, 1):0.00	(2, 1, 1):0.00	(2, 0, 2):0.00	(2, 2, 2):-5.47	(2, 0, 3):0.00	(2, 3, 3):0.00	(2, 1, 0):-4.89	(2, 1, 2):0.00	(3, 0, 0):-4.10	(3, 0, 1):0.00	(3, 1, 1):0.00	(3, 0, 2):-1.36	(3, 2, 2):-5.47	(3, 0, 3):1.86	(3, 3, 3):-8.17	(3, 1, 0):0.00	(3, 1, 2):0.00


* This is enough information to compute $\frac{\partial}{\partial x_j}f(x)$ for all paths
* ...Except that doing that would still not solve all our issues :-(